In [1]:
# Custom Imports
from polymer_utils import analysis, general, filetree, molutils
from polymer_utils import simulation as polysim

from polymer_utils import charging
from polymer_utils.charging.types import AtomIDMap, ResidueChargeMap
from polymer_utils.charging.residues import ChargedResidue

from polymer_utils import LOGGERS_MASTER
from polymer_utils.representation import Polymer, PolymerManager
from polymer_utils.solvation.solvents import WATER_TIP3P
from polymer_utils.analysis import plotprops, trajectory
from polymer_utils.logutils import config_mlf_handler, MultiStreamFileHandler
from polymer_utils.simulation import SimulationPaths, SimulationParameters
from polymer_utils.graphics import rdkdraw

# General Imports
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# Typing and Subclassing
from typing import Any, Callable, ClassVar, Iterable, Optional, Union
from dataclasses import dataclass, field
from abc import ABC, abstractmethod, abstractproperty
from openmm.unit import Unit, Quantity

# File I/O
from pathlib import Path
import csv, json, pickle
from shutil import copyfile, rmtree

# Logging and Shell
import logging
logging.basicConfig(level=logging.INFO)
                            
# Cheminformatics
from rdkit import Chem
from rdkit.Chem import rdmolfiles

# Molecular Dynamics
from openff.units import unit
from openff.interchange import Interchange

from openff.toolkit import ForceField
from openff.toolkit.topology import Topology
from openff.toolkit.topology.molecule import Molecule, Atom
from openff.toolkit.typing.engines.smirnoff.parameters import LibraryChargeHandler

from openmm.openmm import MonteCarloBarostat
from openff.toolkit.utils.exceptions import ConformerGenerationError
from openff.toolkit.utils.toolkits import RDKitToolkitWrapper, OpenEyeToolkitWrapper, AmberToolsToolkitWrapper

from openmm import LangevinMiddleIntegrator, Context
from openmm.vec3 import Vec3
from openmm.app import Simulation, PDBReporter, StateDataReporter

from openmm.unit import picosecond, femtosecond, nanosecond # time
from openmm.unit import nanometer, angstrom # length
from openmm.unit import kelvin, atmosphere # misc

# Static Paths
RESOURCE_PATH = Path('resources')
COLL_PATH = Path('Collections')
# COMPAT_PDB_PATH = Path('compatible_pdbs')
COMPAT_PDB_PATH = Path('compatible_pdbs_updated')

<frozen importlib._bootstrap_external>:572: DeprecationWarning: find_module() is deprecated and slated for removal in Python 3.12; use find_spec() instead
<frozen importlib._bootstrap_external>:1523: DeprecationWarning: FileFinder.find_loader() is deprecated and slated for removal in Python 3.12; use find_spec() instead
<frozen importlib._bootstrap>:283: DeprecationWarning: the load_module() method is deprecated and slated for removal in Python 3.12; use exec_module() instead


## Configuring and (re)loading polymers, setting solvents, checking validity

In [2]:
reset      = False
purge_sims = False 
purge_logs = True # False

poly_source_path = COMPAT_PDB_PATH / 'simple_polymers'
# poly_source_path = COMPAT_PDB_PATH / 'water_soluble_small'
solv_template    = RESOURCE_PATH/'inp_templates'/'solv_polymer_template_box.inp'
desired_solvents = (WATER_TIP3P,) # (None,)
exclusion = 1.0*nanometer

# Define derived paths and create manager
collection_path  = COLL_PATH / poly_source_path.name
structure_path   = poly_source_path / f'{poly_source_path.name}_structures'
monomer_path     = poly_source_path / f'{poly_source_path.name}_monomers'

mgr = PolymerManager(collection_path)

# Perform manager setup / purge actions
if purge_logs: # NOTE : must be done BEFORE log FileHandler is created, as this will destroy it's output as well
    mgr.purge_logs(really=True)

creation_logger = logging.getLogger('polymer_setup')
loggers = [creation_logger, *LOGGERS_MASTER]
logfile_path = mgr.log_dir/f'Setup_{general.timestamp_now()}.log'

with MultiStreamFileHandler(logfile_path, loggers=loggers, proc_name=f'Creation of collection "{mgr.collection_dir.name}"'):
    if reset:
        mgr.purge_collection(really=True, purge_logs=False) # Explicitly DON'T purge logs here (will be done prior to entering log loop)

    if purge_sims:
        mgr.purge_sims(really=True)

    if not mgr.mol_dirs: # will be empty if not yet instantiated or if reset prior
        mgr.populate_collection(struct_dir=structure_path, monomer_dir=monomer_path)
        mgr.solvate_collection(desired_solvents, template_path=solv_template, exclusion=exclusion)

INFO:8772883945106:Creation of collection "simple_polymers" completed in 0:00:00.000027



In [3]:
# Selecting subset of molecules which is suitable for ABE10 charging and subsequent simulation
HARD_POLYMERS = ['vulcanizedrubber', 'polyphenylenesulfone', 'polyethylene', 'polyphenyleneI'] # pathological or otherwise difficult-to-run polymers that I've encountered
EASY_POLYMERS = ['polyvinylchloride_solv_water', 'polyethylmethacrylate_solv_water'] + ['naturalrubber_solv_water', 'PEO_PLGA_solv_water'] # the quickest-to-charge and -to-simulate polymers in the sample set

MolFilter = Callable[[Polymer], bool]
whitelisted  = lambda mol_dir : mol_dir.base_mol_name not in HARD_POLYMERS
matchable    = lambda mol_dir : mol_dir.has_monomer_data
AM1_sized    = lambda mol_dir : 0 < mol_dir.n_atoms <= 300
good_solvent = lambda mol_dir : mol_dir.solvent in desired_solvents

filters : tuple[MolFilter] = (whitelisted, matchable, AM1_sized, good_solvent)
valid_mols = {
    mol_dir.mol_name : mol_dir
        for mol_dir in mgr.mol_dirs_list
            if all(_filter(mol_dir) for _filter in filters)
}

# display to check that loading has gone as planned
print(mgr.all_completed_sims)
print(valid_mols.keys())

{'polyvinylchloride_solv_water': [PosixPath('Collections/simple_polymers/polyvinylchloride/polyvinylchloride_solv_water/MD/04-25-2023_at_18-10-01_PM'), PosixPath('Collections/simple_polymers/polyvinylchloride/polyvinylchloride_solv_water/MD/04-26-2023_at_08-37-42_AM'), PosixPath('Collections/simple_polymers/polyvinylchloride/polyvinylchloride_solv_water/MD/04-26-2023_at_08-59-39_AM'), PosixPath('Collections/simple_polymers/polyvinylchloride/polyvinylchloride_solv_water/MD/04-26-2023_at_08-36-46_AM')], 'naturalrubber_solv_water': [PosixPath('Collections/simple_polymers/naturalrubber/naturalrubber_solv_water/MD/04-25-2023_at_18-13-51_PM')]}
dict_keys(['polymethylketone_solv_water', 'polyvinylchloride_solv_water', 'atactic_styrene_solv_water', 'syntactic_styrene_solv_water', 'naturalrubber_solv_water', 'polythiophene_solv_water', 'polyphenyleneII_solv_water', 'polyethylmethacrylate_solv_water', 'PEO_PLGA_solv_water'])


## Charge and sim loop V2

In [4]:
# DEFINE TARGET MOLECULES AND FORCEFIELD
# sample_mols = EASY_POLYMERS
sample_mols = ['polyvinylchloride_solv_water']
# sample_mols = valid_mols
main_ff_xml = RESOURCE_PATH/'force_fields'/'openff_constrained-2.0.0.offxml'
# main_ff_xml = RESOURCE_PATH/'force_fields'/'openff_unconstrained-2.0.0.offxml'

# CHARGING PARAMETERS
perform_charging = False #True
overwrite_ff_xml     = False #True
overwrite_chg_json   = False #True
distrib_mono_charges = True

strict   = True
verbose  = False

charge_methods = ['ABE10_exact', 'Espaloma_AM1BCC', 'ABE10_averaged'] # NOTE : !CRUCIAL! that 'ABE10_averaged' come last! (need pure charges to averaged from first)
avg_charge_method = 'ABE10_exact'  # method on which to base average charge calculations

# SIMULATION PARAMETERS
run_sims = True
sim_param_path = RESOURCE_PATH/'sim_templates'/'debug_sim.json'
# sim_param_path = RESOURCE_PATH/'sim_templates'/'half_standard_sim.json'
# sim_param_path = RESOURCE_PATH/'sim_templates'/'standard_sim.json'
sim_params = SimulationParameters.from_file(sim_param_path)

# ANALYSIS PARAMETERS
perform_analysis = False #True
traj_sample_interval = 5 # interval between consecutive time points for trajectory analysis (needed for longer sims)

In [5]:
# PRE-FLIGHT CALCULATIONS
if avg_charge_method == 'ABE10_averaged':
    raise ValueError('Charge averaging must be performed over a non-averaged (i.e. pure) set of charges')

sample_dirs = {
    mol_name : mgr.mol_dirs[mol_name] # TOSELF : deliberately not using .get() so this raises an easier-to-debug KeyError
        for mol_name in sample_mols
}

step_names = {
    'charging' : perform_charging,
    'simulation' : run_sims,
    'analysis' : perform_analysis
}
action_str = ' & '.join([
    step_name
        for step_name, perform_step in step_names.items()
            if perform_step
])

print(sim_params)
print(sim_params.num_steps, sim_params.record_freq, action_str)
print(sample_dirs.keys())

SimulationParameters(total_time=Quantity(value=1, unit=picosecond), num_samples=500, charge_method='ABE10_exact', reported_state_data={'step': True, 'time': True, 'potentialEnergy': True, 'kineticEnergy': True, 'totalEnergy': True, 'temperature': True, 'volume': True, 'density': True, 'progress': False, 'remainingTime': False, 'speed': True, 'elapsedTime': True}, timestep=Quantity(value=1, unit=femtosecond), temperature=Quantity(value=300, unit=kelvin), pressure=Quantity(value=1, unit=atmosphere), friction_coeff=Quantity(value=1, unit=/picosecond), barostat_freq=1)
1000 2 simulation
dict_keys(['polyvinylchloride_solv_water'])


In [6]:
# Functions for performing primary charging, sim, and analysis actions
def obtain_partial_charges(mol_dir : Polymer, main_logger : logging.Logger, chg_methods : list[str], avg_charge_method : str, overwrite_chg_json : bool, strict : bool, verbose : bool) -> dict[str, Molecule]:
    '''Ensure a Polymer has all partial charge sets'''
    # 0) LOAD MOLECULE AND TOPOLOGY, ATTEMPT TO APPLY LIBRARY CHARGES
    if not mol_dir.has_monomer_data:
        raise FileExistsError(f'No monomer JSONs found for {mol_dir.mol_name}')

    # 1) ENSURING CHARGES AND RELATED FILES FOR ALL CHARGING METHODS EXIST
    cmols = {}
    for chg_method in chg_methods:
        chgr = charging.application.CHARGER_REGISTRY[chg_method]()

        if chg_method == 'ABE10_averaged': # !NOTE! - critical that this not be the first key in the registry (has nothing to average over from scratch)
            main_logger.info(f'Averaging charges over {mol_dir.mol_name} residues')
            cmol = cmols[avg_charge_method] # obtain the pure charged molecule from which to draw charges for averaging
            avgd_res, atom_id_mapping = charging.averaging.get_averaged_charges(cmol, monomer_data=mol_dir.monomer_data)
            residue_charges = {avgd_res.residue_name : avgd_res.charges for avgd_res in avgd_res}
            chgr.set_residue_charges(residue_charges)
            
            if (mol_dir.monomer_file_chgd is None) or overwrite_chg_json: # can only reach this branch if a json is present but isn't identified as charged within the Polymer
                main_logger.warning('Generating monomer JSON file with monomer-averaged charges')
                mol_dir.create_charged_monomer_file(residue_charges)

                monomer_data = mol_dir.monomer_data_charged # double check that the charged data is in fact loadable
                main_logger.info('Acquired charge-averaged monomer JSON')

        cmol = mol_dir.assert_charges_for(chgr, strict=strict, verbose=verbose)
        cmols[chg_method] = cmol
    
    return cmols

def perform_prop_analysis(mol_dir : Polymer, main_logger : logging.Logger, traj_sample_interval : int):
    '''Analyze trajectories to obtain polymer property data'''
    # aqcuire files for all information
    sim_folder = mol_dir.newest_sim_dir
    main_logger.info(f'Acquiring trajectory data from {sim_folder}')
    sim_paths = SimulationPaths.from_file(next(sim_folder.glob('*_paths.json')))

    sim_params = SimulationParameters.from_file(sim_paths.sim_params)
    state_data = pd.read_csv(sim_paths.state_data)
    traj = trajectory.load_traj(sim_paths.trajectory, topo_path=mol_dir.structure_file, sample_interval=traj_sample_interval, remove_solvent=True)

    # save and plot RDF data
    main_logger.info('Determining pairwise radial distribution functions')
    rdf_dataframe = trajectory.acquire_rdfs(traj, max_rad=1.0*nanometer)
    rdf_dataframe.to_csv(sim_folder/'rdfs.csv')
    rdf_fig, rdf_ax = plotprops.plot_rdfs(rdf_dataframe, scale=15.0)
    rdf_fig.suptitle(f'Pairwise Radial Distribution Functions - {mol_dir.mol_name}')
    rdf_fig.savefig(sim_folder/f'RDFs.png', bbox_inches='tight')
    plt.close()

    # save and plot property data
    main_logger.info('Determining polymer shape properties')
    prop_dataframe = trajectory.acquire_time_props(traj, properties=analysis.polyprops.DEFAULT_PROPS, time_points=sim_params.time_points[::traj_sample_interval]) 
    prop_dataframe.to_csv(sim_folder/'time_series.csv')
    prop_fig, prop_ax = plotprops.plot_time_props(prop_dataframe, scale=18.0)
    prop_fig.suptitle(f'Polymer Shape Properties - {mol_dir.mol_name}')
    prop_fig.savefig(sim_folder/f'shape_props.png', bbox_inches='tight')
    plt.close()
    
    main_logger.info('Successfully exported trajectory analysis data and plots')

In [7]:
# BEGIN CHARGING / SIM LOOP - Perform charge averaging on all target molecules which don't already have averaged LCs; Load forcefield for those which already do 
main_logger = logging.getLogger(__name__)
loggers = [main_logger, *LOGGERS_MASTER] # loggers from all modules which produce logging output
main_logfile_path = mgr.log_dir/f'{action_str.capitalize()}_{general.timestamp_now()}.log'

with MultiStreamFileHandler(main_logfile_path, loggers=loggers, proc_name=f'{action_str.capitalize()} loop') as msf_handler:
    for i, (mol_name, mol_dir) in enumerate(sample_dirs.items()):
        main_logger.info(f'Current molecule: "{mol_name}" ({i + 1}/{len(sample_dirs)})') # +1 converts to more human-readable 1-index for step count
        mol_dir_logfile_path = mol_dir.logs/f'{general.timestamp_now()}.log'
        
        with msf_handler.subhandler(mol_dir_logfile_path, loggers=loggers, proc_name=f'{mol_name} {action_str}') as subhandler: # also log actions to individual Polymers
            if perform_charging:
                main_logger.info('(1) Acquiring partial charges')
                cmols = obtain_partial_charges(mol_dir, main_logger, charge_methods, avg_charge_method, overwrite_chg_json, strict, verbose)
                main_logger.info('') # log empty line to give some breathing room

            if (mol_dir.ff_file is None) or overwrite_ff_xml: # can only reach if a charged monomer json already exists
                main_logger.info('(2) Acquiring Force Field file with Library Charges')
                forcefield, lib_chgs = mol_dir.create_FF_file(xml_src=main_ff_xml, return_lib_chgs=True)
                main_logger.info('') # log empty line to give some breathing room

            if run_sims:
                main_logger.info('(3) Preparing simulation')
                sim_folder = mol_dir.run_simulation_NPT(sim_params)
                main_logger.info('') # log empty line to give some breathing room
            
            if perform_analysis:
                main_logger.info('(4) Analyzing trajectories')
                perform_prop_analysis(mol_dir, main_logger, traj_sample_interval)
                main_logger.info('') # log empty line to give some breathing room

INFO:__main__:Current molecule: "polyvinylchloride_solv_water" (1/1)
INFO:__main__:(3) Preparing simulation
INFO:polymer_utils.representation:Created new Simulation directory "04-26-2023_at_09-01-53_AM"
INFO:polymer_utils.representation:Loading OpenFF Topology WITH monomer graph match
INFO:polymer_utils.representation:Assigning charges from ABE10_exact to polyvinylchloride_solv_water's OpenFF Molecule
INFO:polymer_utils.representation:Creating SMIRNOFF Interchange for "polyvinylchloride_solv_water"
INFO:polymer_utils.simulation:Creating OpenMM Simulation from Interchange
INFO:polymer_utils.simulation:Running 1 ps OpenMM sim at 300 K and 1 atm for 1000 steps
INFO:__main__:
INFO:8772883796266:polyvinylchloride_solv_water simulation completed in 0:00:36.277594

INFO:8772884329298:polyvinylchloride_solv_water simulation completed in 0:00:36.277594

INFO:8772884329298:Simulation loop completed in 0:00:36.282277



In [8]:
msf_handler.children

{}

In [10]:
subhandler.parent

<MultiStreamFileHandler /home/timber/Documents/Python/openff-workspace/polymer_workspace/Collections/simple_polymers/Logs/Simulation_04-26-2023_at_09-01-53_AM.log (NOTSET)>